In [ ]:
# %pip install -r crop_requirments.txt

In [ ]:
# %%capture
# %pip install "unstructured[all-docs]"

In [ ]:
# # Warning control
# import warnings
# warnings.filterwarnings('ignore')

In [ ]:
from IPython.display import JSON

import json

from unstructured.partition.html import partition_html
from unstructured.partition.pdf import partition_pdf
from unstructured.partition.image import partition_image
from unstructured.staging.base import dict_to_elements, elements_to_json

In [ ]:
poppler_path = r"C:/Users/Hemant.Singhsidar/Downloads/Release-24.08.0-0/poppler-24.08.0/Library/bin"

In [ ]:
filename = "Images/page_2.png"

In [ ]:
from unstructured.partition.image import partition_image

elements = partition_image(filename=filename,
                         infer_table_structure=True,
                         strategy='hi_res',
                        #  EXTRACT_TABLE_BLOCK_CROP_HORIZONTAL_PAD = 20,
                        #  EXTRACT_TABLE_BLOCK_CROP_VERTICAL_PAD = 10
                         
           )

In [ ]:
len(elements)

In [ ]:
import json

element_dict = [el.to_dict() for el in elements]
output = json.dumps(element_dict, indent=2)
print(output)

unique_types = set()

for item in element_dict:
    unique_types.add(item['type'])

print(unique_types)

# crop table method 1

In [ ]:
import os
import cv2
import json

if isinstance(output, str):
    output = json.loads(output)

def crop_and_save(image_path, coordinates, output_folder, filename):
    """Crop the specified coordinates from the image and save it."""
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error loading image: {image_path}")
        return
    
    x_min = int(min(pt[0] for pt in coordinates))
    y_min = int(min(pt[1] for pt in coordinates))
    x_max = int(max(pt[0] for pt in coordinates))
    y_max = int(max(pt[1] for pt in coordinates))
    
    cropped_image = image[y_min:y_max, x_min:x_max]
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, filename)
    cv2.imwrite(output_path, cropped_image)
    print(f"Saved: {output_path}")

if isinstance(output, list):
    for item in output:
        if isinstance(item, dict) and item.get("type") in ["Table", "Image"]:
            try:
                image_path = os.path.join(item["metadata"]["file_directory"], item["metadata"]["filename"])
                coordinates = item["metadata"]["coordinates"]["points"]
                output_folder = f"cropped_{item['type'].lower()}"
                filename = f"{item['element_id']}.png"
                crop_and_save(image_path, coordinates, output_folder, filename)
            except KeyError as e:
                print(f"Missing key {e} in item: {item}")
else:
    print("Error: 'output' is not a list of dictionaries.")

In [ ]:
import os
import cv2

def crop_with_padding(image_path, coordinates, output_folder, filename, top_left_padding, bottom_right_padding):
    """Crop the specified coordinates from the image, apply padding, and save it."""
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error loading image: {image_path}")
        return

    x_min = int(min(pt[0] for pt in coordinates))
    y_min = int(min(pt[1] for pt in coordinates))
    x_max = int(max(pt[0] for pt in coordinates))
    y_max = int(max(pt[1] for pt in coordinates))

    # Apply top-left and bottom-right padding
    x_min_padded = x_min - top_left_padding
    y_min_padded = y_min - top_left_padding
    x_max_padded = x_max + bottom_right_padding * 2
    y_max_padded = y_max + bottom_right_padding

    # Ensure padded values stay within the image boundaries
    x_min_padded = max(0, x_min_padded)
    y_min_padded = max(0, y_min_padded)
    x_max_padded = min(image.shape[1], x_max_padded)
    y_max_padded = min(image.shape[0], y_max_padded)

    # Crop the image
    cropped_image = image[y_min_padded:y_max_padded, x_min_padded:x_max_padded]

    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, filename)
    cv2.imwrite(output_path, cropped_image)
    print(f"Cropped image saved to: {output_path}")

# Example usage
table_counter = {}

if isinstance(output, list):
    for item in output:
        if isinstance(item, dict) and item.get("type") == "Table":
            try:
                image_path = os.path.join(item["metadata"]["file_directory"], item["metadata"]["filename"])
                coordinates = item["metadata"]["coordinates"]["points"]
                page_number = item["metadata"].get("page_number", 1)
                
                # Count tables per page
                if page_number not in table_counter:
                    table_counter[page_number] = 1
                else:
                    table_counter[page_number] += 1
                table_number = table_counter[page_number]
                
                output_folder = "Cropped Tables"
                filename = f"Page {page_number}_Table {table_number}.png"
                
                crop_with_padding(image_path, coordinates, output_folder, filename, top_left_padding=5, bottom_right_padding=7)
            except KeyError as e:
                print(f"Missing key {e} in item: {item}")
else:
    print("Error: 'output' is not a list of dictionaries.")

In [ ]:
# import torch
# import numpy as np
# import torch.nn.functional as F
# import onnxruntime as rt
# import cv2

# def keepratio_resize(img):
#     cur_ratio = img.shape[1] / float(img.shape[0])
#     mask_height = 32
#     mask_width = 804
#     if cur_ratio > float(mask_width) / mask_height:
#         cur_target_height = mask_height
#         cur_target_width = mask_width
#     else:
#         cur_target_height = mask_height
#         cur_target_width = int(mask_height * cur_ratio)
#     img = cv2.resize(img, (cur_target_width, cur_target_height))
#     mask = np.zeros([mask_height, mask_width, 3]).astype(np.uint8)
#     mask[:img.shape[0], :img.shape[1], :] = img
#     img = mask
#     return img

# img = cv2.imread('image.png') # 请在替换本地测试图片路径
# img = keepratio_resize(img)
# img = torch.FloatTensor(img)
# chunk_img = []
# for i in range(3):
#     left = (300 - 48) * i
#     chunk_img.append(img[:, left:left + 300, :])
# merge_img = torch.cat(chunk_img, 0)
# data = merge_img.view(3, 32, 300, 3) / 255.
# data = data.permute(0, 3, 1, 2).to('cpu')
# input_data = data.numpy()

# # inference
# sess = rt.InferenceSession('model.onnx')
# input_name = sess.get_inputs()[0].name
# output_name= sess.get_outputs()[0].name
# res = sess.run([output_name], {input_name: input_data})
# outprobs = F.softmax(torch.tensor(res[0]), dim=-1)
# preds = torch.argmax(outprobs, -1)

# # load dict and CTC decode
# # vocab.txt可从模型主页下载
# labelMapping = dict()
# with open('vocab_eng 1.txt', 'r', encoding='utf-8') as f:
#     lines = f.readlines()
#     cnt = 2
#     for line in lines:
#         line = line.strip('\n')
#         labelMapping[cnt] = line
#         cnt += 1

# batchSize, length = preds.shape
# final_str_list = []
# for i in range(batchSize):
#     pred_idx = preds[i].cpu().data.tolist()
#     last_p = 0
#     str_pred = []
#     for p in pred_idx:
#         if p != last_p and p != 0:
#             str_pred.append(labelMapping[p])
#         last_p = p
#     final_str = ''.join(str_pred)
#     final_str_list.append(final_str)

# print(final_str_list)

# Make annotations

In [ ]:
import os
import cv2

def annotate_image_with_rectangle(image_path, coordinates, output_folder, filename, annotation_text, table_count, image_count):
    """Draw rectangle for specified coordinates on the image, annotate, and save it."""
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error loading image: {image_path}")
        return

    x_min = int(max(pt[0] for pt in coordinates))
    y_min = int(min(pt[1] for pt in coordinates))
    x_max = int(min(pt[0] for pt in coordinates))
    y_max = int(max(pt[1] for pt in coordinates)) 

    # Draw rectangle on the image
    color = (255, 0, 0)  # Blue rectangle
    thickness = 2
    top_left_padding = 5  # Adjust this value to control padding size
    Bottom_right_padding = 7  # Adjust this value to control padding size
    x_min_padded = x_min + Bottom_right_padding*2
    y_min_padded = y_min - top_left_padding
    x_max_padded = x_max - top_left_padding
    y_max_padded = y_max + Bottom_right_padding

    # Make sure the padded values don't go out of the image boundaries
    x_min_padded = max(0, x_min_padded)
    y_min_padded = max(0, y_min_padded)
    x_max_padded = min(image.shape[1], x_max_padded)
    y_max_padded = min(image.shape[0], y_max_padded)

    cv2.rectangle(image, (x_min_padded, y_min_padded), (x_max_padded, y_max_padded), color, thickness)

    # Annotate with text
    if annotation_text:
        font = cv2.FONT_HERSHEY_SIMPLEX
        text_position = (x_min, y_min - 10 if y_min - 10 > 10 else y_min + 20)
        cv2.putText(image, annotation_text, text_position, font, 0.5, color, 1, cv2.LINE_AA)

    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, filename)
    cv2.imwrite(output_path, image)
    print(f"Saved: {output_path}")

if isinstance(output, list):
    table_count = 0
    image_count = 0

    for item in output:
        if isinstance(item, dict) and item.get("type") in ["Table", "Image"]:
            try:
                image_path = os.path.join(item["metadata"]["file_directory"], item["metadata"]["filename"])
                coordinates = item["metadata"]["coordinates"]["points"]
                output_folder = f"annotated_{item['type'].lower()}"
                
                if item.get("type") == "Table":
                    table_count += 1
                    filename = f"table_{table_count}.png"
                elif item.get("type") == "Image":
                    image_count += 1
                    filename = f"image_{image_count}.png"

                annotation_text = item.get("type", "")
                annotate_image_with_rectangle(image_path, coordinates, output_folder, filename, annotation_text, table_count, image_count)
            except KeyError as e:
                print(f"Missing key {e} in item: {item}")
else:
    print("Error: 'output' is not a list of dictionaries.")


# Crop table method 2

In [ ]:
import os
import cv2

def extract_blocks(output_data, image_base_path, output_dir):
    """Crop specified coordinates from the image and save them."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    block_count = 0
    
    for element in output_data:
        if element["type"] in ["Table", "Text"]:  # Process only tables and text
            bbox_points = element["metadata"]["coordinates"]["points"]
            filename = os.path.basename(element["metadata"]["filename"])
            
            x_min, y_min = map(round, bbox_points[0])
            x_max, y_max = map(round, bbox_points[2])
            
            image_path = os.path.join(image_base_path, filename)
            im = cv2.imread(image_path)
            if im is None:
                print(f"Error loading image: {image_path}")
                continue
            
            cropped_image = im[y_min:y_max, x_min:x_max]
            block_count += 1
            output_filename = os.path.join(output_dir, f"block_{block_count}.png")
            cv2.imwrite(output_filename, cropped_image)
            print(f"Block {block_count} saved to {output_filename}")

# Example usage:
image_base_path = "Images\page_2.png"
extract_blocks(output, image_base_path, "TESTING/Extracted Tables")


In [ ]:
import json

# Convert elements to dictionary format
elements_dict = [el.to_dict() for el in elements]  # Assuming `elements` is a list of objects

# Target types to extract
target_types = {"Table", "Image"}

# Dictionary to store bounding boxes
bounding_boxes = {}

# Counter for total extracted boxes
total_boxes = 0

for item in elements_dict:
    if item["type"] in target_types:
        # Extract bounding box coordinates
        points = item["metadata"]["coordinates"]["points"]
        
        # Calculate bounding box as [x_min, y_min, x_max, y_max]
        x_min = min(p[0] for p in points)
        y_min = min(p[1] for p in points)
        x_max = max(p[0] for p in points)
        y_max = max(p[1] for p in points)
        
        # Store bounding box using element_id for reference
        bounding_boxes[item["element_id"]] = {
            "type": item["type"],
            "bbox": [x_min, y_min, x_max, y_max],
            "page_number": item["metadata"]["page_number"],
            "filename": item["metadata"]["filename"]
        }
        
        total_boxes += 1

# Print total count of bounding boxes
print(f"Total bounding boxes extracted: {total_boxes}")

# Print bounding boxes data
print(json.dumps(bounding_boxes, indent=2))

In [ ]:
bounding_boxes

In [ ]:
tables = [el for el in elements if el.category == "Table"]

print(tables[0].text)
print(tables[0].metadata.text_as_html)

In [ ]:
tables

In [ ]:
len(tables)

In [ ]:
table_html = tables[2].metadata.text_as_html

In [ ]:
import pandas as pd

# Convert HTML table to pandas DataFrame
dfs = pd.read_html(table_html)
dfs